# Branch and Bround

We are going to: Make a branch and bound algo.

Something where we can make assignments of entities into templates where there are:
- constraints about what entities can fit into what parts of the templates
- constraints about what entities can appear in the same template

Things we need:
- Good representation of the problem
- A way to find valid next moves
- A way to score complete solutiosn
- A way to calc a lowerbound for partial solutions

## Define the problem

Chocolates: 
- Viennese
- Alpini
- Coffee

LinearGiftBox:
- 1 V, 1 A, 1 C
- over all length

Chocolates:
- Type
- Width

Supply:
- List[Chocolates]

Problems:
- 1 box
- supply of chocolates
- assign chocolates to box.
- minimise empty space in box.

Constraints:
- sum(width) in box is less than box over all length
- chocolates must be in the right place
- chocolates can't be in two places at once
- a single space can't contain two chocolates
- chocolates must be in the right slots.


In [763]:
from typing import List, Union
from copy import deepcopy
import math

In [764]:
class Chocolate:
    """Class for a chocolate"""
    id: str
    type: str
    width: int


In [765]:
class ChocolateBox:
    """Class for a chocolate box"""
    
    template: List[str]
    total_width: int
    
    def __init__(self, template, total_width):
        self.num_chocolates = len(template)
        self.template = template
        self.chocolates = [None] * self.num_chocolates
        self.total_width = total_width
        self.remaining_width = total_width
    
    def find_empty_spaces(self) -> List[int]:
        empty_spaces = [idx for idx, choc in enumerate(self.chocolates) if choc is None]
        return empty_spaces

    def identify_possible_chocolates(self, chocolates: List[Chocolate], space_index: int) -> List[Chocolate]:
        chocolate_type = self.template[space_index]
        available_chocolates = [choc for choc in chocolates if choc.type == chocolate_type]
        return available_chocolates
    
    def assign_chocolates(self, chocolate: Chocolate, space: int):
        self.chocolates[space] = chocolate
    
    def calculate_remaining_width(self) -> int:
        populated_weights = [chocolate.width for chocolate in self.chocolates if chocolate is not None]
        remaining_width = self.total_width -sum(populated_weights)
        self.remaining_width = remaining_width
        return self.remaining_width
    
    def calculate_current_width(self) -> int:
        current_total_width = sum([chocolate.width for chocolate in self.chocolates if chocolate is not None])
        return current_total_width
    
    def calculate_remaining_spaces(self) -> int:
        remaining_spaces = len(self.find_empty_spaces())
        return remaining_spaces
    
    def calculate_lower_bound(self) -> int:
        remaining_spaces = self.calculate_remaining_spaces()
        total_spaces = len(self.template)
        lower_bound = math.ceil((self.total_width/ total_spaces)*(total_spaces-remaining_spaces))
        return lower_bound
    
    def calculate_upper_bound(self) -> int:
        lower_bound = self.calculate_lower_bound()
        upper_bound = math.ceil(1.33 * lower_bound)

        return upper_bound
        


In [766]:
class PartialSolution:
    """Class for a partial solution"""
    chocolate_box: ChocolateBox
    available_chocolates: List[Chocolate]

In [767]:
# Define the Viennese chocolates

v_1 = Chocolate()
v_1.id = "v_1"
v_1.type = "viennese"
v_1.width = 1

v_2 = Chocolate()
v_2.id = "v_2"
v_2.type = "viennese"
v_2.width = 2

v_3 = Chocolate()
v_3.id = "v_3"
v_3.type = "viennese"
v_3.width = 3

v_4 = Chocolate()
v_4.id = "v_4"
v_4.type = "viennese"
v_4.width = 4

v_5 = Chocolate()
v_5.id = "v_5"
v_5.type = "viennese"
v_5.width = 5

In [768]:
# Define the Alpini chocolates

a_1 = Chocolate()
a_1.id = "a_1"
a_1.type = "alpini"
a_1.width = 1

a_2 = Chocolate()
a_2.id = "a_2"
a_2.type = "alpini"
a_2.width = 2

a_3 = Chocolate()
a_3.id = "a_3"
a_3.type = "alpini"
a_3.width = 3

a_4 = Chocolate()
a_4.id = "a_4"
a_4.type = "alpini"
a_4.width = 4

a_5 = Chocolate()
a_5.id = "a_5"
a_5.type = "alpini"
a_5.width = 5

In [769]:
# Define the Coffee chocolates

c_1 = Chocolate()
c_1.id = "c_1"
c_1.type = "coffee"
c_1.width = 1

c_2 = Chocolate()
c_2.id = "c_2"
c_2.type = "coffee"
c_2.width = 2

c_3 = Chocolate()
c_3.id = "c_3"
c_3.type = "coffee"
c_3.width = 3

c_4 = Chocolate()
c_4.id = "c_4"
c_4.type = "coffee"
c_4.width = 4

c_5 = Chocolate()
c_5.id = "c_5"
c_5.type = "coffee"
c_5.width = 5

In [770]:
# Define the branch function
def branch(partial_solution: PartialSolution) -> List[PartialSolution]:
    
    empty_spaces = partial_solution.chocolate_box.find_empty_spaces()
    first_empty_space = empty_spaces[0]
    assignable_chocolates = partial_solution.chocolate_box.identify_possible_chocolates(partial_solution.available_chocolates, first_empty_space)
    new_branches = []

    for chocolate in assignable_chocolates:
        new_box = deepcopy(partial_solution.chocolate_box)
        new_box.assign_chocolates(chocolate, first_empty_space)
        new_available_chocolates = partial_solution.available_chocolates.copy()
        new_available_chocolates.remove(chocolate)
        new_partial_solution = PartialSolution()
        new_partial_solution.chocolate_box = new_box
        new_partial_solution.available_chocolates = new_available_chocolates
        new_branches.append( 
        new_partial_solution
        )
    
    return new_branches




In [771]:
# Define the bounding function

def bound(partial_solutions: List[PartialSolution]):
    viable_solutions = []
    for partial_solution in partial_solutions:
    
        # if partial_solution.chocolate_box.calculate_current_width() in range(partial_solution.chocolate_box.calculate_lower_bound(), partial_solution.chocolate_box.calculate_upper_bound()):
        if partial_solution.chocolate_box.calculate_current_width() <= partial_solution.chocolate_box.calculate_lower_bound():
            viable_solutions.append(partial_solution)
    return viable_solutions


In [772]:
# Define the function to prioritise the potential partial solutions

def prioritise_partial_solutions(partial_solutions: List[PartialSolution]) -> List[PartialSolution]:
    ordered_viable_solutions = sorted(partial_solutions, key=lambda x: x.chocolate_box.calculate_current_width(), reverse=True)
    return ordered_viable_solutions


In [773]:
# Define the function to calculate the remaining required iterations over the partial solutions

def calculate_required_iterations_in_for_loop(partial_solution_object: PartialSolution, previous_level: Union[None, PartialSolution]) -> int:
    if previous_level is None:
        return partial_solution_object.chocolate_box.calculate_remaining_spaces()
    else:
        return previous_level.chocolate_box.calculate_remaining_spaces()



In [774]:
# Define the problem

chocolates = [v_1, v_2, v_3, v_4, v_5, a_1, a_2, a_3, a_4, a_5, c_1, c_2, c_3, c_4, c_5 ]
template = ["viennese","alpini", "coffee"]
total_width = 7
chocolate_box = ChocolateBox(template, total_width)

partial_solution = PartialSolution()
partial_solution.chocolate_box = chocolate_box
partial_solution.available_chocolates = chocolates


pending_solutions = []
final_solutions = []
previous_level = None

while True:
   
    required_iterations = calculate_required_iterations_in_for_loop(partial_solution, previous_level)
    for node_level in range(0, required_iterations):
        
        if previous_level is None:
            new_branches = branch(partial_solution)

        else:
            new_branches = branch(previous_level)

        viable_solutions = bound(new_branches)
        ordered_partial_solutions = prioritise_partial_solutions(viable_solutions)
        previous_level = ordered_partial_solutions[0]

        if len(ordered_partial_solutions) > 1:
            pending_solutions += ordered_partial_solutions[1:]
    

    if ordered_partial_solutions[0] not in final_solutions:
        final_solutions.append(ordered_partial_solutions[0])
    
    if bool(pending_solutions):
        previous_level = pending_solutions[0]
        pending_solutions.pop(0)
    
    else:
        break

for idx, sol in enumerate(final_solutions):
    print(f"Solution {idx+1}")
    print(final_solutions[idx].chocolate_box.chocolates[0].id)
    print(final_solutions[idx].chocolate_box.chocolates[1].id)
    print(final_solutions[idx].chocolate_box.chocolates[2].id)


Solution 1
v_3
a_2
c_2
Solution 2
v_2
a_3
c_2
Solution 3
v_1
a_4
c_2
Solution 4
v_3
a_1
c_3
Solution 5
v_2
a_2
c_3
Solution 6
v_2
a_1
c_4
Solution 7
v_1
a_3
c_3
Solution 8
v_1
a_2
c_4
Solution 9
v_1
a_1
c_5
